In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_resumes(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        # Здесь необходимо определить, как извлечь данные резюме из HTML-структуры страницы
        # Например:
        resumes = []
        for resume in soup.find_all('div', class_='resume'):
            name = resume.find('h2').text
            position = resume.find('p', class_='position').text
            experience = int(resume.find('span', class_='experience').text.split()[0])
            age = int(resume.find('span', class_='age').text.split()[0])
            education = resume.find('span', class_='education').text
            skills = resume.find('p', class_='skills').text.split(', ')
            resumes.append({
                'name': name,
                'position': position,
                'experience': experience,
                'age': age,
                'education': education,
                'skills': skills
            })
        return pd.DataFrame(resumes)
    else:
        print(f'Error fetching resumes: {response.status_code}')
        return None

# Пример использования
url = 'https://trudvsem.ru/cv/search'  # Убедитесь, что URL соответствует странице с резюме
data = fetch_resumes(url)
if data is not None:
    data.to_csv('resumes.csv', index=False)


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas as pd

# Загрузка данных
data = pd.read_csv('resumes.csv')

# Преобразование текстовых данных
le_position = LabelEncoder()
data['position_encoded'] = le_position.fit_transform(data['position'])

le_education = LabelEncoder()
data['education_encoded'] = le_education.fit_transform(data['education'])

# Нормализация числовых признаков
scaler = StandardScaler()
data[['experience', 'age']] = scaler.fit_transform(data[['experience', 'age']])

# Формирование признаков и целевой переменной
X = data[['position_encoded', 'experience', 'age', 'education_encoded']]
y = data['skills']  # Предполагается, что 'skills' — целевая переменная

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


EmptyDataError: No columns to parse from file

In [3]:
import tensorflow as tf

# Определение модели
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(len(set(y_train)), activation='softmax')  # Количество классов равно количеству уникальных навыков
])

# Компиляция модели
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


NameError: name 'X_train' is not defined

In [4]:
import streamlit as st
import numpy as np

# Загрузка модели и энкодеров
# model = tf.keras.models.load_model('model.h5')
# le_position = ...
# le_education = ...
# scaler = ...

st.title('Рекомендательная система резюме')

# Ввод данных пользователем
position = st.text_input('Должность')
experience = st.number_input('Опыт работы (лет)', min_value=0)
age = st.number_input('Возраст', min_value=18)
education = st.selectbox('Образование', le_education.classes_)

if st.button('Найти резюме'):
    # Предобработка введённых данных
    position_encoded = le_position.transform([position])[0]
    education_encoded = le_education.transform([education])[0]
    experience_scaled = scaler.transform([[experience, age]])[0][0]
    age_scaled = scaler.transform([[experience, age]])[0][1]

    # Формирование входного вектора
    input_vector = np.array([[position_encoded, experience_scaled, age_scaled, education_encoded]])

    # Предсказание
    predictions = model.predict(input_vector)
    top_indices = predictions.argsort()[0][-5:][::-1]  # Индексы топ-5 рекомендаций

    # Отображение результатов
    st.write('Рекомендуемые резюме:')
    for idx in top_indices:
        st.write(f'Резюме {idx + 1}:')
        st.write(data.iloc[idx])


2024-12-24 14:12:41.337 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-24 14:12:41.423 
  command:

    streamlit run /Users/mihailgoncharov/opt/anaconda3/lib/python3.8/site-packages/ipykernel_launcher.py [ARGUMENTS]
2024-12-24 14:12:41.424 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-24 14:12:41.424 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-24 14:12:41.425 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-24 14:12:41.425 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-24 14:12:41.426 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-24 14:12:41.427 S

NameError: name 'le_education' is not defined

In [5]:
response = requests.get(base_url, params=params, headers=headers)

# Сохраняем содержимое страницы в файл для анализа
with open("page.html", "w", encoding="utf-8") as f:
    f.write(response.text)


NameError: name 'base_url' is not defined